In [1]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import requests
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
# Display all of the columns
pd.set_option('display.max_columns', None)

In [2]:
# Raw URL of the database file on GitHub
url = 'https://github.com/BFletchall/Project-4-Group-4-Machine-Learning/raw/main/claimed_time.db'
# Download the database file
r = requests.get(url)
with open('claimed_time.db', 'wb') as f:
    f.write(r.content)
# Connect to the SQLite database
conn = sqlite3.connect('claimed_time.db')
# Query data from the database table
query = "SELECT * FROM claimed_table"
# Read data into DataFrame
df = pd.read_sql(query, conn)
# Close the database connection
conn.close()
# Display the first few rows of the DataFrame
df.head()

,NumberWithDash,EPM_ProductionControlItemID,EPM_MainMark,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalHours_ThisLaborGroup
0,19-048,870038,T124,1,S.S. TUBE STEEL,2.81,39.575,4.43,HSS,4 x 2 x 0.25,N - No Paint,1,1,Fab/Layout,142,0.0
1,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0
2,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,47,0.0
3,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0
4,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,48,0.0


In [3]:
df.columns

Index(['NumberWithDash', 'EPM_ProductionControlItemID', 'EPM_MainMark',
       'EPM_InstanceNumber', 'Assembly_MainPieceProductionCode',
       'Assembly_MainPartLengthFt', 'Assembly_WeightEachLbs',
       'Assembly_SurfaceAreaEachSqFt', 'Assembly_MainPartShape',
       'Assembly_MainPartDimension', 'Assembly_MainPartFinishDescr',
       'Assembly_TotalQuantityInJob', 'Assembly_NumSmallParts',
       'EPM_AdjustedStationName', 'TimeInSeconds_ThisWorkSegment',
       'Assembly_EstTotalHours_ThisLaborGroup'],
      dtype='object')

In [4]:
# Convert Assembly_EstTotalHours_ThisLaborGroup from hours to seconds
#df['Assembly_EstTotalSeconds_ThisLaborGroup'] = df['Assembly_EstTotalHours_ThisLaborGroup'] * 3600
#df['Assembly_EstTotalSeconds_ThisLaborGroup'] = df['Assembly_EstTotalHours_ThisLaborGroup'].apply(lambda x: x * 3600)

# Fill NaN values in Assembly_EstTotalHours_ThisLaborGroup with 0
df['Assembly_EstTotalHours_ThisLaborGroup'].fillna(0, inplace=True)

# Convert Assembly_EstTotalHours_ThisLaborGroup from hours to seconds
df['Assembly_EstTotalSeconds_ThisLaborGroup'] = df['Assembly_EstTotalHours_ThisLaborGroup'] * 3600

df

,NumberWithDash,EPM_ProductionControlItemID,EPM_MainMark,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalHours_ThisLaborGroup,Assembly_EstTotalSeconds_ThisLaborGroup
0,19-048,870038,T124,1,S.S. TUBE STEEL,2.81,39.575,4.43,HSS,4 x 2 x 0.25,N - No Paint,1,1,Fab/Layout,142,0.0,0.0
1,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0,0.0
2,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,47,0.0,0.0
3,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0,0.0
4,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,48,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8402,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Fab/Layout,16751,0.0,0.0
8403,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Material Handling,17,0.0,0.0
8404,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Weld,4153,0.0,0.0
8405,24-500,904279,S73A,1,PLATE,9.92,94.903,25.14,PL,3/16 x 15,"D - Steel, Paint All",1,0,Fab/Layout,9562,0.0,0.0


In [5]:
#df.dtypes
df['Assembly_EstTotalSeconds_ThisLaborGroup'].unique()

array([    0.     ,  6098.46   , 10474.86   , ...,  2792.673  ,
        2043.96525,   803.72025])

In [6]:
df['Assembly_EstTotalSeconds_ThisLaborGroup'] = df['Assembly_EstTotalSeconds_ThisLaborGroup'].replace(0., np.nan)
df

,NumberWithDash,EPM_ProductionControlItemID,EPM_MainMark,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalHours_ThisLaborGroup,Assembly_EstTotalSeconds_ThisLaborGroup
0,19-048,870038,T124,1,S.S. TUBE STEEL,2.81,39.575,4.43,HSS,4 x 2 x 0.25,N - No Paint,1,1,Fab/Layout,142,0.0,NaN
1,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0,NaN
2,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,47,0.0,NaN
3,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0,NaN
4,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,48,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8402,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Fab/Layout,16751,0.0,NaN
8403,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Material Handling,17,0.0,NaN
8404,24-500,876966,HR103,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Weld,4153,0.0,NaN
8405,24-500,904279,S73A,1,PLATE,9.92,94.903,25.14,PL,3/16 x 15,"D - Steel, Paint All",1,0,Fab/Layout,9562,0.0,NaN


In [7]:
# Drop columns with null values in the column Assembly_EstTotalHours_ThisLaborGroup
#df_drop = df.dropna(axis='columns', subset=['Assembly_EstTotalHours_ThisLaborGroup'])
df_drop = df.dropna(subset=['Assembly_EstTotalSeconds_ThisLaborGroup'])
df_drop

,NumberWithDash,EPM_ProductionControlItemID,EPM_MainMark,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalHours_ThisLaborGroup,Assembly_EstTotalSeconds_ThisLaborGroup
416,22-130,637120,B456,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,62,1.694017,6098.460000
417,22-130,637120,B456,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Peddi Assembler,2525,2.909683,10474.860000
418,22-130,637120,B456,2,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,99,1.694017,6098.460000
419,22-130,637120,B456,2,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Peddi Assembler,2620,2.909683,10474.860000
420,22-130,637120,B456,3,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,14,1.694017,6098.460000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7354,24-011,837518,B107,38,CHANNEL,12.00,261.570,40.31,C,6 x 10.5,N - No Paint,45,11,Weld,1649,0.099224,357.207843
7355,24-011,837518,B107,39,CHANNEL,12.00,261.570,40.31,C,6 x 10.5,N - No Paint,45,11,Weld,4,0.049612,178.603922
7366,24-011,868657,B109,1,CHANNEL,11.08,125.442,26.59,C,4 x 5.4,N - No Paint,8,10,Fab/Layout,2310,0.775743,2792.673000
7367,24-011,868657,B109,1,CHANNEL,11.08,125.442,26.59,C,4 x 5.4,N - No Paint,8,10,Material Handling,241,0.567768,2043.965250


In [10]:
df_reduce =df_drop.drop(columns=['EPM_MainMark', 'Assembly_EstTotalHours_ThisLaborGroup', 'NumberWithDash'], inplace=False)
df_reduce.head(30)

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalSeconds_ThisLaborGroup
416,637120,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,62,6098.460
417,637120,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Peddi Assembler,2525,10474.860
418,637120,2,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,99,6098.460
419,637120,2,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Peddi Assembler,2620,10474.860
420,637120,3,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Material Handling,14,6098.460
421,637120,3,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",3,18,Peddi Assembler,2614,10474.860
422,637131,1,BEAM,23.08,633.943,101.31,W,12 x 26,"DS - Special Case/Steel, Paint All",1,8,Material Handling,95,24469.200
423,637131,1,BEAM,23.08,633.943,101.31,W,12 x 26,"DS - Special Case/Steel, Paint All",1,8,Peddi Assembler,2322,7328.304
424,637137,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",1,18,Material Handling,20,6098.652
425,637137,1,BEAM,40.97,2293.910,257.74,W,21 x 55,"DS - Special Case/Steel, Paint All",1,18,Peddi Assembler,2589,10475.028


In [11]:
df_reduce.columns

Index(['EPM_ProductionControlItemID', 'EPM_InstanceNumber',
       'Assembly_MainPieceProductionCode', 'Assembly_MainPartLengthFt',
       'Assembly_WeightEachLbs', 'Assembly_SurfaceAreaEachSqFt',
       'Assembly_MainPartShape', 'Assembly_MainPartDimension',
       'Assembly_MainPartFinishDescr', 'Assembly_TotalQuantityInJob',
       'Assembly_NumSmallParts', 'EPM_AdjustedStationName',
       'TimeInSeconds_ThisWorkSegment',
       'Assembly_EstTotalSeconds_ThisLaborGroup'],
      dtype='object')

In [12]:
# Determine the number of unique values in each column.
unique_values = df_reduce.nunique()
print(unique_values)

EPM_ProductionControlItemID                 827
EPM_InstanceNumber                           90
Assembly_MainPieceProductionCode             18
Assembly_MainPartLengthFt                   476
Assembly_WeightEachLbs                      738
Assembly_SurfaceAreaEachSqFt                724
Assembly_MainPartShape                       10
Assembly_MainPartDimension                  156
Assembly_MainPartFinishDescr                  9
Assembly_TotalQuantityInJob                  19
Assembly_NumSmallParts                       74
EPM_AdjustedStationName                       6
TimeInSeconds_ThisWorkSegment              1598
Assembly_EstTotalSeconds_ThisLaborGroup    1954
dtype: int64


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
df_reduce_encoded = pd.get_dummies(df_reduce)
df_reduce_encoded

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalSeconds_ThisLaborGroup,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPartShape_AC,Assembly_MainPartShape_C,Assembly_MainPartShape_FB,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_W,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1/2 x 5,Assembly_MainPartDimension_1/2 x 7,Assembly_MainPartDimension_1/4 x 1,Assembly_MainPartDimension_1/4 x 104 §,Assembly_MainPartDimension_1/4 x 13 1/2,Assembly_MainPartDimension_1/4 x 13 1/2 §,Assembly_MainPartDimension_1/4 x 15 1/8 §,Assembly_MainPartDimension_1/4 x 17 3/8 §,Assembly_MainPartDimension_1/4 x 18 1/2,Assembly_MainPartDimension_1/4 x 18 1/2 §,Assembly_MainPartDimension_1/4 x 2,Assembly_MainPartDimension_1/4 x 25,Assembly_MainPartDimension_1/4 x 27 §,Assembly_MainPartDimension_1/4 x 6,Assembly_MainPartDimension_1/4 x 6 §,Assembly_MainPartDimension_10 3/4 x 0.25,Assembly_MainPartDimension_10 x 10 x 0.375,Assembly_MainPartDimension_10 x 100,Assembly_MainPartDimension_10 x 12,Assembly_MainPartDimension_10 x 15.3,Assembly_MainPartDimension_10 x 33,Assembly_MainPartDimension_10 x 39,Assembly_MainPartDimension_10 x 4 x 0.375,Assembly_MainPartDimension_10 x 45,Assembly_MainPartDimension_10 x 49,Assembly_MainPartDimension_10 x 6 x 0.375,Assembly_MainPartDimension_10 x 77,Assembly_MainPartDimension_12 3/4 x 0.25,Assembly_MainPartDimension_12 x 14.3,Assembly_MainPartDimension_12 x 170,Assembly_MainPartDimension_12 x 20.7,Assembly_MainPartDimension_12 x 22,Assembly_MainPartDimension_12 x 26,Assembly_MainPartDimension_12 x 40,Assembly_MainPartDimension_12 x 53,Assembly_MainPartDimension_12 x 58,Assembly_MainPartDimension_12 x 65,Assembly_MainPartDimension_12 x 72,Assembly_MainPartDimension_12 x 79,Assembly_MainPartDimension_12 x 8.639,Assembly_MainPartDimension_12 x 96,Assembly_MainPartDimension_14 x 120,Assembly_MainPartDimension_14 x 159,Assembly_MainPartDimension_14 x 211,Assembly_MainPartDimension_14 x 22,Assembly_MainPartDimension_14 x 233,Assembly_MainPartDimension_14 x 283,Assembly_MainPartDimension_14 x 30,Assembly_MainPartDimension_14 x 342,Assembly_MainPartDimension_14 x 398,Assembly_MainPartDimension_14 x 455,Assembly_MainPartDimension_14 x 500,Assembly_MainPartDimension_14 x 550,Assembly_MainPartDimension_14 x 61,Assembly_MainPartDimension_14 x 68,Assembly_MainPartDimension_14 x 82,Assembly_MainPartDimension_14 x 90,Assembly_MainPartDimension_16 x 26,Assembly_MainPartDimension_16 x 31,Assembly_MainPartDimension_18 x 35,Assembly_MainPartDimension_18 x 40,Assembly_MainPartDimension_18 x 46,Assembly_MainPartDimension_18 x 97,Assembly_MainPartDimension_2 x 2 x 0.25,Assembly_MainPartDimension_20 x 8 x 0.625,Assembly_MainPartDimension_21 x 44,Assembly_MainPartDimension_21 x 50,Assembly_MainPartDimension_21 x 55,Assembly_MainPartDimension_24 x 55,Assembly_MainPartDimension_24 x 62,Assembly_MainPartDimension_24 x 68,Assembly_MainPartDimension_24 x

In [14]:
df_reduce_encoded.columns

Index(['EPM_ProductionControlItemID', 'EPM_InstanceNumber',
       'Assembly_MainPartLengthFt', 'Assembly_WeightEachLbs',
       'Assembly_SurfaceAreaEachSqFt', 'Assembly_TotalQuantityInJob',
       'Assembly_NumSmallParts', 'TimeInSeconds_ThisWorkSegment',
       'Assembly_EstTotalSeconds_ThisLaborGroup',
       'Assembly_MainPieceProductionCode_ALUM STRINGER',
       ...
       'Assembly_MainPartFinishDescr_ES - Special Case/Exterior Exposed',
       'Assembly_MainPartFinishDescr_G - Galvanized',
       'Assembly_MainPartFinishDescr_GF - Fabrication Req'd After Galvanizing',
       'Assembly_MainPartFinishDescr_N - No Paint',
       'EPM_AdjustedStationName_Beam Cambering', 'EPM_AdjustedStationName_Cut',
       'EPM_AdjustedStationName_Fab/Layout',
       'EPM_AdjustedStationName_Material Handling',
       'EPM_AdjustedStationName_Peddi Assembler',
       'EPM_AdjustedStationName_Weld'],
      dtype='object', length=208)

In [15]:
X = df_reduce_encoded.copy()
X.drop("Assembly_SurfaceAreaEachSqFt", axis=1, inplace=True)
X.head()

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalSeconds_ThisLaborGroup,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPartShape_AC,Assembly_MainPartShape_C,Assembly_MainPartShape_FB,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_W,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1/2 x 5,Assembly_MainPartDimension_1/2 x 7,Assembly_MainPartDimension_1/4 x 1,Assembly_MainPartDimension_1/4 x 104 §,Assembly_MainPartDimension_1/4 x 13 1/2,Assembly_MainPartDimension_1/4 x 13 1/2 §,Assembly_MainPartDimension_1/4 x 15 1/8 §,Assembly_MainPartDimension_1/4 x 17 3/8 §,Assembly_MainPartDimension_1/4 x 18 1/2,Assembly_MainPartDimension_1/4 x 18 1/2 §,Assembly_MainPartDimension_1/4 x 2,Assembly_MainPartDimension_1/4 x 25,Assembly_MainPartDimension_1/4 x 27 §,Assembly_MainPartDimension_1/4 x 6,Assembly_MainPartDimension_1/4 x 6 §,Assembly_MainPartDimension_10 3/4 x 0.25,Assembly_MainPartDimension_10 x 10 x 0.375,Assembly_MainPartDimension_10 x 100,Assembly_MainPartDimension_10 x 12,Assembly_MainPartDimension_10 x 15.3,Assembly_MainPartDimension_10 x 33,Assembly_MainPartDimension_10 x 39,Assembly_MainPartDimension_10 x 4 x 0.375,Assembly_MainPartDimension_10 x 45,Assembly_MainPartDimension_10 x 49,Assembly_MainPartDimension_10 x 6 x 0.375,Assembly_MainPartDimension_10 x 77,Assembly_MainPartDimension_12 3/4 x 0.25,Assembly_MainPartDimension_12 x 14.3,Assembly_MainPartDimension_12 x 170,Assembly_MainPartDimension_12 x 20.7,Assembly_MainPartDimension_12 x 22,Assembly_MainPartDimension_12 x 26,Assembly_MainPartDimension_12 x 40,Assembly_MainPartDimension_12 x 53,Assembly_MainPartDimension_12 x 58,Assembly_MainPartDimension_12 x 65,Assembly_MainPartDimension_12 x 72,Assembly_MainPartDimension_12 x 79,Assembly_MainPartDimension_12 x 8.639,Assembly_MainPartDimension_12 x 96,Assembly_MainPartDimension_14 x 120,Assembly_MainPartDimension_14 x 159,Assembly_MainPartDimension_14 x 211,Assembly_MainPartDimension_14 x 22,Assembly_MainPartDimension_14 x 233,Assembly_MainPartDimension_14 x 283,Assembly_MainPartDimension_14 x 30,Assembly_MainPartDimension_14 x 342,Assembly_MainPartDimension_14 x 398,Assembly_MainPartDimension_14 x 455,Assembly_MainPartDimension_14 x 500,Assembly_MainPartDimension_14 x 550,Assembly_MainPartDimension_14 x 61,Assembly_MainPartDimension_14 x 68,Assembly_MainPartDimension_14 x 82,Assembly_MainPartDimension_14 x 90,Assembly_MainPartDimension_16 x 26,Assembly_MainPartDimension_16 x 31,Assembly_MainPartDimension_18 x 35,Assembly_MainPartDimension_18 x 40,Assembly_MainPartDimension_18 x 46,Assembly_MainPartDimension_18 x 97,Assembly_MainPartDimension_2 x 2 x 0.25,Assembly_MainPartDimension_20 x 8 x 0.625,Assembly_MainPartDimension_21 x 44,Assembly_MainPartDimension_21 x 50,Assembly_MainPartDimension_21 x 55,Assembly_MainPartDimension_24 x 55,Assembly_MainPartDimension_24 x 62,Assembly_MainPartDimension_24 x 68,Assembly_MainPartDimension_24 x 76,Assembly_MainPartDimensio

In [16]:
y = df_reduce_encoded['TimeInSeconds_ThisWorkSegment'].ravel()
y[:5]

array([  62, 2525,   99, 2620,   14], dtype=int64)

In [17]:
# Preview the data
print("Features Shape:",X.shape)
print("Targe Shape:", y.shape)

Features Shape: (2794, 207)
Targe Shape: (2794,)


In [18]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [20]:
# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

In [21]:
X_scaler

StandardScaler()

In [22]:
# Scaling data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [66]:
X_test_scaled

array([[-0.52387025, -0.26612276,  1.44584396, ..., -0.75139899,
        -0.19135187,  1.60280844],
       [ 0.27227413, -0.26612276,  1.09402051, ...,  1.33085086,
        -0.19135187, -0.62390488],
       [ 1.63915227, -0.25348802, -0.06870318, ..., -0.75139899,
        -0.19135187, -0.62390488],
       ...,
       [-0.52387025, -0.26612276,  1.44584396, ..., -0.75139899,
        -0.19135187, -0.62390488],
       [-0.07892644, -0.26612276,  2.1153648 , ..., -0.75139899,
        -0.19135187, -0.62390488],
       [ 0.6142153 , -0.26612276,  0.55937887, ...,  1.33085086,
        -0.19135187, -0.62390488]])

# Fitting the Random Forest Model

In [23]:
# Instantiate the Random Forest Regression model
rf_reg = RandomForestRegressor()

In [24]:
# Fit the model to the training data
rf_reg.fit(X_train_scaled, y_train)

RandomForestRegressor()

In [25]:
# Predict the target values for the test set
y_pred_rf = rf_reg.predict(X_test_scaled)

In [26]:
# Evaluate the model
print("Random Forest Regression Model Score (R^2):", rf_reg.score(X_test_scaled, y_test))
print("Random Forest Regression Mean Absolute Error:", np.mean(np.abs(y_pred_rf - y_test)))

Random Forest Regression Model Score (R^2): 0.7978171439591115
Random Forest Regression Mean Absolute Error: 460.17386266094434


In [27]:
# Display feature importances
feature_importances = rf_reg.feature_importances_
print("Feature Importances:")
for i, feature in enumerate(X.columns):
    print(f"{feature}: {feature_importances[i]}")

Feature Importances:
EPM_ProductionControlItemID: 0.005799176911604251
EPM_InstanceNumber: 8.040793063103419e-07
Assembly_MainPartLengthFt: 0.0001121444017586446
Assembly_WeightEachLbs: 0.00342350245591902
Assembly_TotalQuantityInJob: 2.348707018523264e-06
Assembly_NumSmallParts: 0.0005148031585438283
TimeInSeconds_ThisWorkSegment: 0.6727264543865505
Assembly_EstTotalSeconds_ThisLaborGroup: 0.30706284168368636
Assembly_MainPieceProductionCode_ALUM STRINGER: 5.878340507794167e-11
Assembly_MainPieceProductionCode_ANGLE: 4.4543047724180265e-09
Assembly_MainPieceProductionCode_BEAM: 1.5416764933589617e-05
Assembly_MainPieceProductionCode_BENT PLATE: 3.5620410296148983e-09
Assembly_MainPieceProductionCode_BOLLARD: 3.1310086940195854e-11
Assembly_MainPieceProductionCode_CHANNEL: 1.7211573783851987e-08
Assembly_MainPieceProductionCode_COLUMN: 2.0757915590385882e-05
Assembly_MainPieceProductionCode_CRANE GIRDER: 0.0011593952445549303
Assembly_MainPieceProductionCode_EMBED: 1.1977564830367396e-

# Predict and Plot

In [71]:
# Predict the target values for the test set
y_pred_rf = rf_reg.predict(X_test_scaled)

# Create a DataFrame to store actual and predicted values
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_rf}, index=X_test.index)

# Add columns from X_test to add feature detail
predictions = pd.concat([predictions, X_test], axis=1)

# Add another column
predictions['Difference'] = predictions['Actual'] - predictions['Predicted']

predictions.head(15)

,Actual,Predicted,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalSeconds_ThisLaborGroup,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPartShape_AC,Assembly_MainPartShape_C,Assembly_MainPartShape_FB,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_W,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1/2 x 5,Assembly_MainPartDimension_1/2 x 7,Assembly_MainPartDimension_1/4 x 1,Assembly_MainPartDimension_1/4 x 104 §,Assembly_MainPartDimension_1/4 x 13 1/2,Assembly_MainPartDimension_1/4 x 13 1/2 §,Assembly_MainPartDimension_1/4 x 15 1/8 §,Assembly_MainPartDimension_1/4 x 17 3/8 §,Assembly_MainPartDimension_1/4 x 18 1/2,Assembly_MainPartDimension_1/4 x 18 1/2 §,Assembly_MainPartDimension_1/4 x 2,Assembly_MainPartDimension_1/4 x 25,Assembly_MainPartDimension_1/4 x 27 §,Assembly_MainPartDimension_1/4 x 6,Assembly_MainPartDimension_1/4 x 6 §,Assembly_MainPartDimension_10 3/4 x 0.25,Assembly_MainPartDimension_10 x 10 x 0.375,Assembly_MainPartDimension_10 x 100,Assembly_MainPartDimension_10 x 12,Assembly_MainPartDimension_10 x 15.3,Assembly_MainPartDimension_10 x 33,Assembly_MainPartDimension_10 x 39,Assembly_MainPartDimension_10 x 4 x 0.375,Assembly_MainPartDimension_10 x 45,Assembly_MainPartDimension_10 x 49,Assembly_MainPartDimension_10 x 6 x 0.375,Assembly_MainPartDimension_10 x 77,Assembly_MainPartDimension_12 3/4 x 0.25,Assembly_MainPartDimension_12 x 14.3,Assembly_MainPartDimension_12 x 170,Assembly_MainPartDimension_12 x 20.7,Assembly_MainPartDimension_12 x 22,Assembly_MainPartDimension_12 x 26,Assembly_MainPartDimension_12 x 40,Assembly_MainPartDimension_12 x 53,Assembly_MainPartDimension_12 x 58,Assembly_MainPartDimension_12 x 65,Assembly_MainPartDimension_12 x 72,Assembly_MainPartDimension_12 x 79,Assembly_MainPartDimension_12 x 8.639,Assembly_MainPartDimension_12 x 96,Assembly_MainPartDimension_14 x 120,Assembly_MainPartDimension_14 x 159,Assembly_MainPartDimension_14 x 211,Assembly_MainPartDimension_14 x 22,Assembly_MainPartDimension_14 x 233,Assembly_MainPartDimension_14 x 283,Assembly_MainPartDimension_14 x 30,Assembly_MainPartDimension_14 x 342,Assembly_MainPartDimension_14 x 398,Assembly_MainPartDimension_14 x 455,Assembly_MainPartDimension_14 x 500,Assembly_MainPartDimension_14 x 550,Assembly_MainPartDimension_14 x 61,Assembly_MainPartDimension_14 x 68,Assembly_MainPartDimension_14 x 82,Assembly_MainPartDimension_14 x 90,Assembly_MainPartDimension_16 x 26,Assembly_MainPartDimension_16 x 31,Assembly_MainPartDimension_18 x 35,Assembly_MainPartDimension_18 x 40,Assembly_MainPartDimension_18 x 46,Assembly_MainPartDimension_18 x 97,Assembly_MainPartDimension_2 x 2 x 0.25,Assembly_MainPartDimension_20 x 8 x 0.625,Assembly_MainPartDimension_21 x 44,Assembly_MainPartDimension_21 x 50,Assembly_MainPartDimension_21 x 55,Assembly_MainPartDimension_24 x 55,Assembly_MainPartDimension_24 x 62,Assembly_MainPartDimension_24 x 68,Assembly_MainPartDimension_24 x 76,Assembly

In [76]:
# Get the column names of the DataFrame
all_columns = predictions_concat.columns

# Filter the column names that start with "Assembly_MainPartDimension"
MainPieceProductionCode_columns = [col for col in all_columns if col.startswith("Assembly_MainPieceProductionCode")]
MainPartShape_columns = [col for col in all_columns if col.startswith("Assembly_MainPartShape")]
MainPartDimension_columns = [col for col in all_columns if col.startswith("Assembly_MainPartDimension")]
MainPartFinishDescr_columns = [col for col in all_columns if col.startswith("Assembly_MainPartFinishDesc")]
AdjustedStationName_columns = [col for col in all_columns if col.startswith("EPM_AdjustedStationName")]


In [88]:
# Lists of specific column names
MainPieceProductionCode_columns
MainPartShape_columns
MainPartDimension_columns
MainPartFinishDescr_columns
AdjustedStationName_columns

# Make a copy of the "predictions" DataFrame
predictions_concat = predictions.copy()

# Function to find the column name where the value is True among specified columns for each unique index

def find_true_MainPieceProductionCode(row):
    for column in MainPieceProductionCode_columns:
        if row[column]:  # Check if the value is True
            return column
    return "No Code"

def find_true_MainPartShape(row):
    for column in MainPartShape_columns:
        if row[column]:  # Check if the value is True
            return column
    return "No Code"

def find_true_MainPartDimension(row):
    for column in MainPartDimension_columns:
        if row[column]:  # Check if the value is True
            return column
    return "No Code"

def find_true_MainPartFinishDescr(row):
    for column in MainPartFinishDescr_columns:
        if row[column]:  # Check if the value is True
            return column
    return "No Code"

def find_true_AdjustedStationName(row):
    for column in AdjustedStationName_columns:
        if row[column]:  # Check if the value is True
            return column
    return "No Code"


# Create a new column "Assemby_MainPieceProductionCodeTrue" containing the column name with "TRUE" value
predictions_concat['Assembly_MainPieceProductionCode'] = predictions_concat.apply(find_true_MainPieceProductionCode, axis=1)
predictions_concat['Assembly_MainPartShape'] = predictions_concat.apply(find_true_MainPartShape, axis=1)
predictions_concat['Assembly_MainPartDimension'] = predictions_concat.apply(find_true_MainPartDimension, axis=1)
predictions_concat['Assembly_MainPartFinishDescr'] = predictions_concat.apply(find_true_MainPartFinishDescr, axis=1)
predictions_concat['EPM_AdjustedStationName'] = predictions_concat.apply(find_true_AdjustedStationName, axis=1)

# Display the updated DataFrame
predictions_concat.head(15)

,Actual,Predicted,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalSeconds_ThisLaborGroup,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPartShape_AC,Assembly_MainPartShape_C,Assembly_MainPartShape_FB,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_W,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1/2 x 5,Assembly_MainPartDimension_1/2 x 7,Assembly_MainPartDimension_1/4 x 1,Assembly_MainPartDimension_1/4 x 104 §,Assembly_MainPartDimension_1/4 x 13 1/2,Assembly_MainPartDimension_1/4 x 13 1/2 §,Assembly_MainPartDimension_1/4 x 15 1/8 §,Assembly_MainPartDimension_1/4 x 17 3/8 §,Assembly_MainPartDimension_1/4 x 18 1/2,Assembly_MainPartDimension_1/4 x 18 1/2 §,Assembly_MainPartDimension_1/4 x 2,Assembly_MainPartDimension_1/4 x 25,Assembly_MainPartDimension_1/4 x 27 §,Assembly_MainPartDimension_1/4 x 6,Assembly_MainPartDimension_1/4 x 6 §,Assembly_MainPartDimension_10 3/4 x 0.25,Assembly_MainPartDimension_10 x 10 x 0.375,Assembly_MainPartDimension_10 x 100,Assembly_MainPartDimension_10 x 12,Assembly_MainPartDimension_10 x 15.3,Assembly_MainPartDimension_10 x 33,Assembly_MainPartDimension_10 x 39,Assembly_MainPartDimension_10 x 4 x 0.375,Assembly_MainPartDimension_10 x 45,Assembly_MainPartDimension_10 x 49,Assembly_MainPartDimension_10 x 6 x 0.375,Assembly_MainPartDimension_10 x 77,Assembly_MainPartDimension_12 3/4 x 0.25,Assembly_MainPartDimension_12 x 14.3,Assembly_MainPartDimension_12 x 170,Assembly_MainPartDimension_12 x 20.7,Assembly_MainPartDimension_12 x 22,Assembly_MainPartDimension_12 x 26,Assembly_MainPartDimension_12 x 40,Assembly_MainPartDimension_12 x 53,Assembly_MainPartDimension_12 x 58,Assembly_MainPartDimension_12 x 65,Assembly_MainPartDimension_12 x 72,Assembly_MainPartDimension_12 x 79,Assembly_MainPartDimension_12 x 8.639,Assembly_MainPartDimension_12 x 96,Assembly_MainPartDimension_14 x 120,Assembly_MainPartDimension_14 x 159,Assembly_MainPartDimension_14 x 211,Assembly_MainPartDimension_14 x 22,Assembly_MainPartDimension_14 x 233,Assembly_MainPartDimension_14 x 283,Assembly_MainPartDimension_14 x 30,Assembly_MainPartDimension_14 x 342,Assembly_MainPartDimension_14 x 398,Assembly_MainPartDimension_14 x 455,Assembly_MainPartDimension_14 x 500,Assembly_MainPartDimension_14 x 550,Assembly_MainPartDimension_14 x 61,Assembly_MainPartDimension_14 x 68,Assembly_MainPartDimension_14 x 82,Assembly_MainPartDimension_14 x 90,Assembly_MainPartDimension_16 x 26,Assembly_MainPartDimension_16 x 31,Assembly_MainPartDimension_18 x 35,Assembly_MainPartDimension_18 x 40,Assembly_MainPartDimension_18 x 46,Assembly_MainPartDimension_18 x 97,Assembly_MainPartDimension_2 x 2 x 0.25,Assembly_MainPartDimension_20 x 8 x 0.625,Assembly_MainPartDimension_21 x 44,Assembly_MainPartDimension_21 x 50,Assembly_MainPartDimension_21 x 55,Assembly_MainPartDimension_24 x 55,Assembly_MainPartDimension_24 x 62,Assembly_MainPartDimension_24 x 68,Assembly_MainPartDimension_24 x 76,Assembly

In [94]:
# Make a copy of the "predictions_concat" DataFrame
predictions_analysis = predictions_concat.copy()

# List of columns to drop
columns_to_drop = (MainPieceProductionCode_columns + 
                   MainPartShape_columns + 
                   MainPartDimension_columns + 
                   MainPartFinishDescr_columns + 
                   AdjustedStationName_columns)

# Drop the specified columns from the DataFrame
predictions_analysis.drop(columns=columns_to_drop, inplace=True)

# Remove the common prefix from the values in the column "Assembly_MainPieceProductionCode"
predictions_analysis['Assembly_MainPieceProductionCode'] =predictions_analysis['Assembly_MainPieceProductionCode'].str.replace('Assembly_MainPieceProductionCode_', '')
predictions_analysis['Assembly_MainPartShape'] =predictions_analysis['Assembly_MainPartShape'].str.replace('Assembly_MainPartShape_', '')
predictions_analysis['Assembly_MainPartDimension'] =predictions_analysis['Assembly_MainPartDimension'].str.replace('Assembly_MainPartDimension_', '')
predictions_analysis['Assembly_MainPartFinishDescr'] =predictions_analysis['Assembly_MainPartFinishDescr'].str.replace('Assembly_MainPartFinishDescr_', '')
predictions_analysis['EPM_AdjustedStationName'] =predictions_analysis['EPM_AdjustedStationName'].str.replace('EPM_AdjustedStationName_', '')

# Display the updated DataFrame
predictions_analysis.head(15)

# Export the DataFrame to a CSV file
predictions_analysis.to_csv('predictions_analysis.csv', index_label='Index')